In [1]:
import sys
sys.path.insert(1, '../')

from river import stream,tree,metrics
from encoding import prefix_bin
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from collections import Counter

from tqdm import tqdm
import pickle as pkl
import matplotlib.pyplot as plt
import os,json
import datetime
from collections import deque, Counter
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

import warnings

warnings.filterwarnings("ignore")
import xgboost as xgb

import gzip

import datetime, time
import utils
import sliding_window
import psutil

import copy

In [2]:
def display_progress(row_counting, total_length, interval=20000):
    if rowcounter%interval == 0:
        print(round(rowcounter*100/totallength,2) ,'%', 'Case finished: %s'%(casecount), 'Running cases: %s'%(len(streaming_db)))

In [3]:
def update_test_cases(new_case, test_cases):
    test_cases.append(new_case)
    if len(test_cases) > test_size:
        test_cases.popleft()

In [4]:
def variant_list(window_data):
    variant_window = {}
    for t in window_data:
        values = [i for i in t.encoded if t.encoded[i]==1]
        activities = str(sorted([i for i in values if 'activity' in i]))

        if activities not in variant_window.keys():
            variant_window[activities] =1
        else:
            variant_window[activities] +=1
    return variant_window

In [5]:
def variant_coverage(train_variant, test_variant):   
    coverage_ratio = 0
    for t in test_variant.keys():
        if t in train_variant.keys():
            coverage_ratio += test_variant[t]

    return coverage_ratio

In [6]:
def label_distribution(test_data):
    testing_label = {'True':0, 'False':0, '':0}
    for t in test_data:
        label = t[maximum_prefix-1].true_label
        testing_label[label]+=1

    label_dist = int(testing_label['True'])/30
    return label_dist


In [7]:
def trigger_cd(dataset_label, cd_type):
    file_path = '../triggered_cd/%s/%s_CD_list.pkl'%(cd_type, dataset_label)
    with open(file_path, 'rb') as f:
        data = pkl.load(f)
    return data

In [9]:
# for dataset_label in ['Production_Data','sepsis_cases_1','sepsis_cases_2','sepsis_cases_3','traffic_fines_1','hospital_billing_1',
#                       'hospital_billing_2']:

# ['bpic2011_1', 'bpic2011_3', 'bpic2011_4', 'bpic2015_1', 'bpic2015_2', 'bpic2015_3', 'bpic2015_4', 'bpic2015_5']
# ['bpic2012_1', 'bpic2012_2', 'bpic2012_3', 'bpic2017_1', 'bpic2017_2', 'bpic2017_3']

for dataset_label in ['bpic2012_1', 'bpic2012_2', 'bpic2012_3', 'bpic2017_1', 'bpic2017_2', 'bpic2017_3']:

    # Experiment settings
    label_condition = False
    cd_type = 'prefixtreeCDD'
    # cd_type = 'prefixtreeCDD'
    
    print(dataset_label)
    './DATA/logs/synthetic_log_bc1.csv'
    
    # Invoke parameters for dataset
    window_size = 200
    test_size = 30
    gp = 200
    training_rebalance = True
    retraining_condition = cd_type
    classifier = 'rf'
    retraining_trigger = False
    with open('../dataset_parameters.json','r') as json_file:
        parameters = json.load(json_file)[dataset_label]
        print(parameters)
        key_pair = parameters['key_pair']
        catatars = parameters['categorical_attrs']
        maximum_prefixs = parameters['maximum_prefix']
        prefix_range = parameters['prefix']

    for maximum_prefix in prefix_range:
        training_time = []
        print(maximum_prefix)
        retraining_check = True
        train_window_dict = {}
        test_window_dict = {}
        dataset_loc = '../DATA/logs/'+ dataset_label +'.csv'
        cd_list = deque(trigger_cd(dataset_label, cd_type))
        try:
            os.makedirs('../result/%s'%(dataset_label))
        except:
            pass

        # Set streaming classifier
        if classifier == 'rf':
            streaming_classifier = RandomForestClassifier(n_estimators = 10, n_jobs = -1, random_state=500)
        elif classifier == 'xgb':
            streaming_classifier = xgb.XGBClassifier(n_estimators = 10, learning_rate=0.01, verbosity =0, random_state=500)     

        dataset = stream.iter_csv(
                dataset_loc
                )

        totallength = len(list(dataset))

        dataset = stream.iter_csv(
                dataset_loc,
                # drop=['Complete Timestamp'],
                target='outcome'
                )
        enctype = 'Index-base'

        streaming_db ={}
        training_windows = sliding_window.training_window(window_size,test_size)
        training_models ={}
        test_cases = deque()
        feature_matrix ={}

        save_model = {}
        casecount = 0
        rowcounter = 0
        finished_db ={}
        running_case = 0
        window_acc_dict = {}
        prediction_result = dict()
        for i in range(1, maximum_prefix+1): prediction_result[i] = {}
        finished_caseid = set()
        usedingrace = set()


        '''
        Before test and training streaming event predictive monitoring, grace period is preceded to initialize model by prefix length
        and obtain feature matrix to transform future events
        '''
        for x,y in dataset:
    #         display_progress(rowcounter, totallength)
            rowcounter +=1
            # Event stream change dictionary keys
            x = utils.dictkey_chg(x, key_pair)

            if dataset_label !='bpic15':
                x['ts'] = x['ts'][:-4]

            x['outcome'] =y 
            # Initialize case by prefix length        
            caseid = x['caseid']
            outcome = x['outcome']
        #     progress = x['progress']

            x.pop('caseid')
            x.pop('outcome')

        #     x.pop('progress')

            case_bin = prefix_bin(caseid, x)
            case_bin.set_enctype(enctype)

            if caseid not in list(streaming_db.keys()):
                case_bin.set_prefix_length(1)    
                streaming_db[caseid] = []
            elif caseid in finished_caseid:
                pass
            else:
                case_bin.set_prefix_length(len(streaming_db[caseid])+1)
                case_bin.set_prev_enc(streaming_db[caseid][-1])

            # Encode event and cases and add to DB
            case_bin.update_truelabel(outcome)   
            case_bin.update_encoded(catattrs=catatars,enctype=enctype)
            ts = case_bin.event['ts']
            streaming_db[caseid].append(case_bin)
            # Detect label appeared case 
            if outcome != '' and caseid not in finished_caseid:
                usedingrace.add(caseid)
                for i in streaming_db[caseid]:
                    i.update_truelabel(outcome)
                finished_caseid.add(caseid)
                # Adding newly finished case to training set.    
                casecount +=1
                # Grace period to collect feature matrix
                if casecount < gp:
                    case_length = len(streaming_db[caseid])
                    if case_length >= maximum_prefix:
                        if 'prefix_%s'%(maximum_prefix) not in list(feature_matrix.keys()):
                            feature_matrix['prefix_%s'%(maximum_prefix)]=set()
                            training_models['prefix_%s'%(maximum_prefix)] = [streaming_classifier,
                                                                       0]
                        feature_list = list(streaming_db[caseid][maximum_prefix-1].encoded.keys())
                        for x in feature_list: feature_matrix['prefix_%s'%(maximum_prefix)].add(x) 
                else:
                    break

        if casecount not in train_window_dict.keys(): train_window_dict[casecount] = []
        if casecount not in test_window_dict.keys(): test_window_dict[casecount] = []

        for caseid in list(usedingrace):
            case_length = len(streaming_db[caseid])
            if case_length >= maximum_prefix:
                x = streaming_db[caseid][maximum_prefix-1]
                if x.prefix_length != 0:            
                    training_windows.update_window(x)

                update_test_cases(streaming_db[caseid], test_cases)
        train_window_dict[casecount].append(copy.deepcopy(training_windows.container))

        training_x = []
        training_y = []
        for pos, i in enumerate(training_windows.container):
            x_prefix_length = i.prefix_length 
            i.encoded = utils.readjustment_training(i.encoded, feature_matrix['prefix_%s'%(maximum_prefix)])
            training_x.append(i.encoded)
            training_y.append(i.true_label)


        le = LabelEncoder()
        training_y = le.fit_transform(training_y)
        training_x = pd.DataFrame.from_dict(training_x)

        ###
        #Oversampling
        ###
        n_labels = Counter(training_y)['True']

        if n_labels <=2:
            pass
        elif n_labels>2 and n_labels <=5:
            smote = SMOTE(k_neighbors=n_labels-1)
            training_x, training_y = smote.fit_resample(training_x, training_y)
        else:
            smote = SMOTE()
            training_x, training_y = smote.fit_resample(training_x, training_y)

        start_time = time.time()
        training_models['prefix_%s'%(x_prefix_length)][0].fit(training_x, training_y)
        training_models['prefix_%s'%(x_prefix_length)][1] =casecount
        end_time = time.time()
        training_time.append(end_time-start_time)
        
        prediction_result[maximum_prefix][casecount] = {}
        y_truelist = []
        y_predlist = []
        for case in test_cases:
            if len(case) >= maximum_prefix:
                x = case[maximum_prefix-1]
                if x.prefix_length != 0:            
                    model = training_models['prefix_%s'%(x_prefix_length)][0]
                    current_event = utils.readjustment_training(x.encoded, feature_matrix['prefix_%s'%(maximum_prefix)])
                    current_event = pd.Series(current_event).to_frame().T
                    y_pred = training_models['prefix_%s'%(x_prefix_length)][0].predict_proba(current_event)
                    y_truelist.append(x.true_label)
                    y_predlist.append(y_pred)
                    test_window_dict[casecount].append(x)
        prediction_result[maximum_prefix][casecount]['y_true'] = y_truelist
        prediction_result[maximum_prefix][casecount]['y_pred'] = y_predlist

        train_variant = variant_list(train_window_dict[200][0])
        save_model[training_models['prefix_%s'%(x_prefix_length)][1]] = training_models['prefix_%s'%(x_prefix_length)][0]
        '''
        Streaming event label prediction start.
        - Test and training steps are executed when case finished/ event arrived with label
        '''
        for i in streaming_db.keys(): usedingrace.add(i)
        streaming_db ={}
        cdhappend ={}
        for i in range(1, maximum_prefix+1): cdhappend[i] = 0

        for x,y in dataset:
            display_progress(rowcounter, totallength)

            rowcounter +=1
            # Event stream change dictionary keys
            x = utils.dictkey_chg(x, key_pair)

            if dataset_label !='bpic15':
                x['ts'] = x['ts'][:-4]

            # Check label possible
            # x = utils.set_label(x)
            x['outcome'] =y 
            # Initialize case by prefix length
            caseid = x['caseid']
            outcome = x['outcome']
            x.pop('caseid')
            x.pop('outcome')

            if caseid not in usedingrace:
                case_bin = prefix_bin(caseid, x)
                case_bin.set_enctype(enctype)

                if caseid not in list(streaming_db.keys()):
                    case_bin.set_prefix_length(1)    
                    streaming_db[caseid] = []
                    running_case +=1
                elif caseid in finished_caseid:
                    pass
                else:
                    case_bin.set_prefix_length(len(streaming_db[caseid])+1)
                    case_bin.set_prev_enc(streaming_db[caseid][-1])

                # Encode event and cases and add to DB
                case_bin.update_truelabel(outcome)   
                case_bin.update_encoded(catattrs=catatars,enctype=enctype)
                ts = case_bin.event['ts']

                if case_bin.prefix_length == maximum_prefix:
                    case_bin.encoded = utils.readjustment_training(case_bin.encoded, feature_matrix['prefix_%s'%(case_bin.prefix_length)])
                streaming_db[caseid].append(case_bin)

                # Detect label appeared case 
                if outcome != '' and caseid not in finished_caseid:
                    finished_caseid.add(caseid)

                    # Adding newly finished case to training set.
                    casecount +=1    

                    # Modify encoded attributes of cases with feature matrix
                    case_length = len(streaming_db[caseid])
                    if case_length >= maximum_prefix:

                        streaming_db[caseid][maximum_prefix-1].update_truelabel(outcome)
                        update_test_cases(streaming_db[caseid], test_cases)
                        x = streaming_db[caseid][maximum_prefix-1].encoded
                        prefix_length =streaming_db[caseid][maximum_prefix-1].prefix_length                    

                        test_variant = variant_list([i[maximum_prefix-1] for i in test_cases])
                        variant_cover = variant_coverage(train_variant, test_variant)/test_size
                        label_dist = label_distribution(test_cases)
                        training_windows.update_window(streaming_db[caseid][maximum_prefix-1])
                        if retraining_condition == 'prodrift' or retraining_condition == 'prefixtreeCDD':
                            if len(cd_list) ==0:
                                retraining_trigger = False
                                pass
                            else:
                                if test_cases[0][0].caseid == cd_list[0]:
                                    retraining_trigger = True
                                    cd_list.popleft()
                                    print('Triggered1')
                                else:
                                    retraining_trigger = False
                        
                        if retraining_trigger == True:
                            print('Triggered2')
                            if casecount not in train_window_dict.keys(): train_window_dict[casecount] = []
                            train_window_dict[casecount].append(copy.deepcopy(training_windows.container))                       

                            x_training = pd.DataFrame.from_dict([i.encoded for i in training_windows.container])
                            for i in x_training.columns.values: x_training[i] = x_training[i].fillna(0)
                            feature_matrix['prefix_%s'%(maximum_prefix)] = x_training.columns.values

                            training_x = []
                            training_y = []
                            for pos, i in enumerate(training_windows.container):
                                x_prefix_length = i.prefix_length 
                                i.encoded = utils.readjustment_training(i.encoded, feature_matrix['prefix_%s'%(maximum_prefix)])
                                training_x.append(i.encoded)
                                training_y.append(i.true_label)
                            training_y = le.fit_transform(training_y)
                            training_x = pd.DataFrame.from_dict(training_x)

                            ###
                            #Oversampling
                            ###
                            n_labels = Counter(training_y)['True']

                            if n_labels <=2:
                                pass
                            elif n_labels>2 and n_labels <=5:
                                smote = SMOTE(k_neighbors=n_labels-1)
                                training_x, training_y = smote.fit_resample(training_x, training_y)
                            else:
                                smote = SMOTE()
                                training_x, training_y = smote.fit_resample(training_x, training_y)
                                                   ###
                            #Model retraining
                            ###
                            start_time = time.time()
                            training_models['prefix_%s'%(maximum_prefix)][0] = streaming_classifier
                            training_models['prefix_%s'%(maximum_prefix)][0].fit(training_x, training_y)
                            training_models['prefix_%s'%(x_prefix_length)][1] =casecount
                            train_variant = variant_list(train_window_dict[casecount][0])
                            end_time = time.time()
                            training_time.append(end_time-start_time)
                            
                            if retraining_condition == 'label':
                                train_window_dict = dict()

                            save_model[training_models['prefix_%s'%(x_prefix_length)][1]] = training_models['prefix_%s'%(x_prefix_length)]
#                             save_model[training_models['prefix_%s'%(x_prefix_length)][1]] = 0
                    y_truelist = []
                    y_predlist = []

                    if casecount not in test_window_dict.keys():
                        test_window_dict[casecount] = []

                    for case in test_cases:
                        if len(case) >= maximum_prefix:
                            x = case[maximum_prefix-1]
                            if x.prefix_length != 0:
                                length = x.prefix_length
                                current_event = utils.readjustment_training(x.encoded, feature_matrix['prefix_%s'%(maximum_prefix)])
                                current_event = pd.Series(current_event).to_frame().T

                                if casecount != gp:
                                    c_id = [x.caseid for x in test_window_dict[casecount-1]]

                                if x.caseid not in c_id:
                                    y_pred = training_models['prefix_%s'%(x_prefix_length)][0].predict_proba(current_event)
                                else:
                                    y_pred = prediction_result[maximum_prefix][casecount-1]['y_pred'][c_id.index(x.caseid)]
                                y_truelist.append(x.true_label)
                                y_predlist.append(y_pred)

                                if casecount not in test_window_dict.keys(): test_window_dict[casecount] = []
                                test_window_dict[casecount].append(x)
                        prediction_result[maximum_prefix][casecount] = {}
                        prediction_result[maximum_prefix][casecount]['y_true'] = y_truelist
                        prediction_result[maximum_prefix][casecount]['y_pred'] = y_predlist
    #                 if 'b1' not in caseid and cdhappend[maximum_prefix] == 0:
    #                     cdhappend[maximum_prefix] = model_update_count

        try:
            os.makedirs('../result/%s/%s/Finished cases/Trigger %s'%(dataset_label, classifier, retraining_condition))
        except:
            pass    

        with gzip.open('../result/%s/%s/Finished cases/Trigger %s/prefix_%s training window retrained.pkl'%(dataset_label, classifier, retraining_condition, maximum_prefix), 'wb') as f:
            pkl.dump(train_window_dict, f)
        with gzip.open('../result/%s/%s/Finished cases/Trigger %s/prefix_%s test window retrained.pkl'%(dataset_label, classifier, retraining_condition, maximum_prefix), 'wb') as f:
            pkl.dump(test_window_dict, f)
        with gzip.open('../result/%s/%s/Finished cases/Trigger %s/prefix_%s update retrained.pkl'%(dataset_label, classifier, retraining_condition, maximum_prefix), 'wb') as f:
            pkl.dump(prediction_result, f)
        with gzip.open('../result/%s/%s/Finished cases/Trigger %s/prefix_%s model.pkl'%(dataset_label, classifier, retraining_condition, maximum_prefix), 'wb') as f:
            pkl.dump(save_model, f)
        with gzip.open('../result/time/%s_%s_%s_%s_trainingtime.pkl'%(dataset_label, classifier, maximum_prefix, retraining_condition), 'wb') as f:
            pkl.dump(training_time, f)

bpic2012_1
{'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Resource': 'resource', 'Complete Timestamp': 'ts'}, 'categorical_attrs': ['activity', 'resource'], 'maximum_prefix': 36, 'prefix': [11, 21, 30]}
11
10.45 % Case finished: 200 Running cases: 66
20.9 % Case finished: 330 Running cases: 598
31.35 % Case finished: 719 Running cases: 1093
41.8 % Case finished: 1204 Running cases: 1538
52.25 % Case finished: 1692 Running cases: 1970
62.7 % Case finished: 2071 Running cases: 2552
73.15 % Case finished: 2525 Running cases: 3019
83.6 % Case finished: 2993 Running cases: 3481
Triggered1
Triggered2
94.05 % Case finished: 3602 Running cases: 3876
21
10.45 % Case finished: 200 Running cases: 66
20.9 % Case finished: 330 Running cases: 598
31.35 % Case finished: 719 Running cases: 1093
41.8 % Case finished: 1204 Running cases: 1538
52.25 % Case finished: 1692 Running cases: 1970
62.7 % Case finished: 2071 Running cases: 2552
73.15 % Case finished: 2525 Running cases: 3019
83.6 % 

In [42]:
def trigger_cd(dataset_label, cd_type):
    file_path = '../triggered_cd/%s/%s_CD_list.pkl'%(cd_type, dataset_label)
    with open(file_path, 'rb') as f:
        data = pkl.load(f)
    return data

In [48]:
trigger_cd('bpic2015_3','prodrift')

[]